$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Final Project: Blackjack Reinforcement Learning

Diego Batres, Devin Dennis, Tanner Nickels

## Introduction

  In this project we aimed to design a series of programs using A.I. techniques to analyze various facets of the casino game Blackjack. The aspect of the game that interested us the most was the Blackjack strategy of card-counting, in which a player keeps track of the cards played to make better guesses about what moves to make in order to maximize profits. Furthermore, we wanted to see if there was a computational way for a third-party to watch a series of games and determine if a player was counting cards or not.

Initially, we intended to train a series of “players” using reinforcement learning and various learning heuristics to play the game. We defined these as random, basic, and counter, corresponding to random decision-making, established basic-strategy, and card-counting respectively. Then, we would attempt to run a linear regression on betting data for basic players in order to predict betting values to try and catch a player that is making decisions based off of card-counting. We hypothesized that random and basic players would converge on a similar playing style after training, but that their betting patterns would be significantly different than those of a player who is counting cards. To set up the problem, we first needed to research the game and the various strategies used for betting and gameplay, as well as the mechanics behind card-counting and the methods used by casinos to catch card-counters. 


Blackjack is a popular casino game offered in most casinos around the world. To start, you and the dealer are each dealt two cards with the object of the game being to beat the dealer. The goal is to receive cards from the dealer one at a time to get as close to 21 points without going over, also called “busting”. Each numbered card is counted at face-value, with face cards counted as 10, and the Ace counted as 1 or 11, depending on which is more beneficial to the player. When it is the player’s turn, they can either hit, stand, double-down, or split cards with the same value. For the purpose of this project, we decided to limit the player only to standing or hitting. While reducing player decisions in this fashion changed the odds of winning, it simplified the training and gameplay aspects of the game. We wanted to simplify the game as much as possible without making too many changes since we were unsure of the feasibility of incorporating all factors in the time allotted, and planned to add more factors to the game later if possible.  We also decided to use a soft-17 system of gameplay for the dealer, meaning the dealer will hit until they reach 17 or higher, with the Ace always being counted as 11 points. 


Since the popularization of card-counting in the 1960’s, casinos have made a concerted effort to discourage and prevent people from using the system. This mathematical strategy of gameplay is designed to give a player just enough of an edge over the casino that they could play the “house” for very large amounts of money. While not illegal, card-counting is highly frowned upon by casino management because of how much of an impact it can have on a player’s winnings. The system involves assigning “counts” to each card, then keeping track of a “running-count” for each card that gets dealt into play. By assigning positive values to low cards like 2 and 3, and negative values to higher cards like King and Ace, over time the running count can indicate whether there are more high cards or more low cards left in the deck, allowing a player to bet and play accordingly. When the count is higher than 5-7, there is a significant chance that they will be dealt blackjack on the next hand, which pays out 3-2, as opposed to the usual 1-1 for all other wins. This is the time when the player has the most edge over the casino. There are dozens of unique strategies that follow this principle, but for simplicity we mainly use the Hi-Lo system where values of -1, 0, and +1 are assigned to the cards. Once we had the parameters of gameplay set up, we were able to begin designing our programs.


## Methods

### Reinforcement Learning

Reinforcement learning is a subdiscipline of machine learning that consists of training a program by the use of positive and negative reinforcements, which are added to a cumulative score to determine the total reward for a defined action. In game playing artificial intelligence, reinforcements are given with each successful outcome of a game, and a negative reinforcement is given with each loss. This data is stored in a Q table, a data structure consisting of state-move tuples as keys and the total reinforcement as the value. When training AI to play a game, the initial choices are made randomly to explore possible actions, and the reinforcement is adjusted based on whether those random actions were successful. 

The next step is decreasing the level of randomness for each iteration using a variable named `epsilon`, which changes at a rate of `epsilonDecayFactor`. The decision to make a random move or use previous knowledge is specified in an `epsilonGreedy` function, which generates a random value and compares it to `epsilon`. `epsilon` starts at `1.0`, meaning that 100% of the choices are random, and with each iteration it decays, eventually converging to `0`. By that time, the goal of the program is to have enough data on previous games in the Q table to completely base its decisions off of previous knowledge. By having explored multiple options it is able to correctly choose the one which will result in a higher reward. The reward is directly influenced by a variable `learningRate`, which specifies how much of a reinforcement you are adding every time. 

For the game of blackjack, we chose to use reinforcement learning because each hand has only 3 possible outcomes: win, draw, or loss. The idea of using reinforcement learning for this game is simple: the state is represented as a player’s hand and the move is either `Hit` or `Stand`. If this choice results in a win, we give a positive reinforcement to the player, and if they lose, we give a negative reinforcement. Since the game is partially based on luck of which cards you were handed, we base our reinforcement off the player’s decision to hit or stand. That way, after playing enough games a player will know when to hit or stand to avoid busting, or hitting a total hand value over the limit of 21. We then use various betting strategies to optimize a player’s reward after winning a game.

### Betting Strategy

In order to really run an effective analysis on the players’ betting patterns, we needed to make sure we included a diverse selection of betting strategies. More importantly, we wanted to ensure that our basic and random players bets diverged enough from the counter to allow for our program to look for differences. 

In the game, there is a term known as a “bet spread”, corresponding to the table-minimum bet and the maximum bet a player is willing to put down. We set this to be around 10-50. Using a smaller bet spread for each player results in all players appearing indistinguishable from one another during gameplay. However, the players would be allowed to go beyond the bet spread at different times to account for a human-factor of betting known as the Gambler’s Fallacy. This is a cognitive bias in which players bet more money if they win more games, despite the fact that games are statistically independent from each other...unless you’re counting cards. Thus, we end up with players that generally bet within a given range, deviating at times if they’re winning more or if they’re counting cards. This was done to make it more difficult for our program to catch the counter. 

The main strategies that we used were random, basic, and counter, reflective of our different players. Later, we added various different strategies for the counter in an effort to test our betting analysis program. The random better simply bets random values within the bet spread. The basic player uses a progressive betting system where the bet is increased by a small amount every time the player wins, and decreased every time they lose, only deviating from the bet spread if they win many games in a small span. Finally, the counter places bets based off the count of the table, also incorporating bet spread, betting spikes, and penetration depending on the technique. All of these will be explained in Implementation. 

### Pearson Correlation and Betting Spikes

Once we had a series of players operating under different betting and playing strategies, it was time for the main goal of our experiment: catching the counter. We decided to use two different methods to analyze this. 

The second method we used was an analysis of betting spikes. This is an event in the game where a player increases their bet by a significantly large amount. For a basic player, this could occur when they decide to leave the table and go all-in on the last hand, when a player is on a “hot streak” and decides to bet big on the next hand, or when the true count of the table gets high enough that a counter decides it would be beneficial to bet big. Our program looks at how much a player increases their bet and whether or not they won the last hand to look for spikes.

The combination of these two methods would allow us to look for the counter. If a player spikes their bet, the program will then check the correlation of their past bets to determine if it’s a fluke event or if the spike was caused by an increase in table count. 

## Implementation

### Deck / Cards

To represent the cards in play, we modified code from ‘Think Python: an Introduction to Software Design’ which consisted of `Hand`, `Deck`, and `Card` objects. We adjusted this code to include additional information and methods required for Blackjack, such as card counts, busting, and multiple decks. The `Card` object was left unchanged. The `Deck` object was changed to keep a running count of cards which is used by card counting players to base their bets off of. This allows card counters to easily access the true count of the cards in play without keeping score in their hand, which is reset every time a game is over and new cards are dealt. The `Hand` object includes additional methods to test whether or not a player has busted, and if they have a blackjack, which involves modifying the value of aces to be 1 or 11 as needed.


In [1]:
import random


class Card(object):
    """represents a standard playing card."""

    suit_names = ["Clubs", "Diamonds", "Hearts", "Spades"]
    rank_names = [None, "Ace", "2", "3", "4", "5", "6", "7",
                  "8", "9", "10", "Jack", "Queen", "King"]

    def __init__(self, suit=0, rank=2):
        self.suit = suit
        self.rank = rank
        self.value = self.get_value()

    def __str__(self):
        return '%s of %s has a value of %s.' % (Card.rank_names[self.rank],
                                                Card.suit_names[self.suit], self.value)

    def __cmp__(self, other):
        t1 = self.suit, self.rank
        t2 = other.suit, other.rank
        return cmp(t1, t2)

    def __lt__(self, other):
        if self.suit < other.suit:
            return True
        elif self.suit > other.suit:
            return False
        else:
            return self.rank < other.rank

    def get_value(self):
        rank = Card.rank_names[self.rank]
        if self.is_ace():
            return 11
        elif rank in ["Jack", "Queen", "King"]:
            return 10
        else:
            return int(rank)

    def is_ace(self):
        return Card.rank_names[self.rank] == "Ace"


class Deck(object):
    """represents a deck of cards"""

    def __init__(self, numDecks=1):
        self.numDecks = numDecks
        self.cards = []
        self.count = 0
        for deck in range(numDecks):
            for suit in range(4):
                for rank in range(1, 14):
                    card = Card(suit, rank)
                    self.cards.append(card)
        self.shuffle()

    def __str__(self):
        res = []
        for card in self.cards:
            res.append(str(card))
        return '\n'.join(res)

    def __len__(self):
        return len(self.cards)

    def add_card(self, card):
        """add a card to the deck"""
        self.cards.append(card)

    def pop_card(self, i=-1):
        """remove and return a card from the deck.
        By default, pop the last card."""
        card = self.cards.pop(i)
        self.count += self.count_card(card)
        
        if self.empty():
            self.reset()

        return card

    def shuffle(self):
        """shuffle the cards in this deck"""
        random.shuffle(self.cards)

    def sort(self):
        """sort the cards in ascending order"""
        self.cards.sort()

    def move_cards(self, hand, num):
        """move the given number of cards from the deck into the Hand"""
        for i in range(num):
            hand.add_card(self.pop_card())

    def count_card(self, card):
        card_value = card.get_value()

        if card_value >= 10 or card.rank is "Ace":
            return -1
        elif card_value < 10 and card_value > 6:
            return 0
        elif card_value <= 6:
            return 1

    def true_count(self):
        return self.count / self.remaining_decks()

    def remaining_decks(self):
        decks = int(round(len(self)/52))
        if decks is 0:
            return 1
        return decks

    def penetration(self):
        return (self.numDecks*52 - len(self.cards)) / (self.numDecks*52)

    def empty(self):
        return len(self.cards)==0
    
    def reset(self):
        
        self.cards.clear()
        self.count = 0
        
        for deck in range(self.numDecks):
            for suit in range(4):
                for rank in range(1, 14):
                    card = Card(suit, rank)
                    self.cards.append(card)
        self.shuffle()
        

class Hand(Deck):
    """represents a hand of playing cards"""

    def __init__(self, label=''):
        self.label = label
        self.cards = []

    def __str__(self):
        for card in self.cards:
            print(card)
        return "Value of hand: " + str(self.get_value())

    def __len__(self):
        return len(self.cards)

    """returns the value for the given Hand"""

    def get_value(self):
        hand_value = 0
        aces = self.get_num_aces()
        for card in self.cards:
            hand_value += card.value
        while (
            hand_value > 21) and aces:  # corner case to adjust value of hand given the number of aces and current value of the player's hand
            hand_value -= 10
            aces -= 1
        return hand_value

    """ helper method for get_value()"""

    def get_num_aces(self):
        num_aces = 0
        for card in self.cards:
            if card.is_ace():
                num_aces += 1
        return num_aces

    """adds card to hand"""

    def add(self, card):
        self.cards.append(card)
        return self.cards

    def bust(self):
        return self.get_value() > 21

    def blackjack(self):
        if len(self) == 2:
            if self.get_num_aces() == 1:
                for card in self.cards:
                    if card.get_value() == 10:
                        return True
        return False
    
    def clear(self):
        self.cards.clear()

### Player Types

In order to test the success of various playing strategies, we implemented three types of players: random, basic, and card counting. The random player starts off by making purely random choices, which eventually become dependent on their Q table with the reinforcements that they received during training. The idea of having a random player is to depend solely on reinforcement learning without any additional information. The basic player is given a matrix consisting of possible hand combinations and the best action to take for each state. This matrix can easily be found online, and corresponds to the statistical outcomes of various states as calculated by statisticians. It is the foundation for what blackjack players call “basic strategy”.

The goal with the basic player was to see if the reinforcement learning deviated from these values, eventually leading the basic player to make decisions without basing itself off the original strategy. Our card counting player was trained using the same strategy as the basic player, however, we employ various betting strategies to differentiate from a basic or random player who has no knowledge of the running count of the cards. Additionally, the counter was given a slight edge, since some hands in basic strategy (in actual card-counting there are 18 hands), require you to deviate from basic for better odds. In these few cases (for us, 10, since we don’t double down or split) the counter will deviate.

An interesting observation in our training results is that all types of players converged on similar values, and these values accurately reflect the true odds of blackjack. In a game of blackjack, it is expected that players win about 42.22% of the time, draw about 8.48%, and lose about 49.10%, before incorporating splitting and doubling down. Our players converged on values within only a few percent of the expected values, supporting our initial hypothesis that random and basic players will end up being the same. What surprised us is that the counter player also converged close to these values, indicating that our training could have likely been accomplished with only one `basicGreedy` function.

### Basic Functions

Our entire training structure was based off of the Towers of Hanoi and Tic-Tac-Toe code provided in class, though we ended up making many modifications to the actual training. Some of our basic functions like `makeMove()`, `validMove()`, `end_of_hand()`, and `winner()` are very self explanatory, offering basic functionality for gameplay. 

In [2]:
# makeMove(): applies move to hand
def makeMove(hand, move, deck):
    if move == 'Hit':
        hand.add_card(deck.pop_card())
    else:
        return
    
# validMoves(): returns a list of valid moves (hit or stand)
def validMoves():
    return ["Hit", "Stand"]

# end_of_hand(): returns true if move marks end of play
def end_of_hand(hand, move):
    if hand.bust():
        return True
    if move is "Stand":
        return True
    return False

# winner(): returns true if p1 beats p2
def winner(p1, p2):
    if p1.blackjack():
        return True
    if p2.bust():
        return True
    return p1.get_value() > p2.get_value()

`hand_to_state()` is our way of converting a `Hand` object into a tuple for Q-table lookup, and `state_dealer_tuple()` is used to get a tuple for basic strategy lookup. 

In [3]:
# hand_to_state(): returns a tuple of the hand's value and number of aces
def hand_to_state(hand):
    value = hand.get_value()
    aces = hand.get_num_aces()
    return (value, aces)

# state_dealer_tuple(): returns a tuple of state and dealer's card
def state_dealer_tuple(state, dealer):
    dealerCard = str(dealer.cards[0].rank)
    if dealerCard is 'Ace':
        dealerCard = 'A'
    elif dealerCard is 'Jack' or dealerCard is 'Queen' or dealerCard is 'King':
        dealerCard = '10'
    return (state, dealerCard)

Some of the more confusing methods we used included `readCardCountingMatrix` and `readBasicMatrix`, both used to read in the different play matrices found online that correspond to basic strategy. They convert a text file into a dictionary with a state/dealer tuple as the key and a move as the value. 

In [4]:
# readBasicMatrix(): reads in decision matrix for basic player
def readBasicMatrix(filename):
    decisions = {}
    dealer = []
    player = []
    index = 0
    with open(filename) as f:
        dealer = f.readline().split()
        player = [(int(entry.split(",")[0][1:]), int(entry.split(",")[1][:1])) for entry in f.readline().split()]
        for line in f:
            vals = line.split()
            for i in range(len(dealer)):
                decisions[(player[index], dealer[i])] = vals[i]
            index += 1
    f.close()
    return decisions

# readCardCountingmatrix(): reads in card counting matrix with card counts and BC/PA/IC values
def readCardCountingMatrix(filename):
    cc = {}
    with open(filename) as f:
        for line in f:
            vals = line.split()
            cc[vals[0]] = [vals[1:11], vals[11:]] 
    return cc

Finally, we needed a simple way to play the dealer’s hand during gameplay. Since the player and dealer dont take turns between cards, this can be accomplished with a single function that gives the dealer cards until they reach a soft-17 or higher, without the built-in Hand function trying to count Aces as 1.

In [5]:
# playDealer(): makes play decisions for dealer (stand on values of 17 or more) 
def play_dealer(dealer, deck):
    value = dealer.get_value()
    while value < 17:
        card = deck.pop_card()
        dealer.add(card)
        value += card.get_value()

We also have functions to deal with file IO when reading or writing text files.

In [6]:
import json
from ast import literal_eval

# writeQ(): write Q table to file
def writeQ(filename, Q):
    strQ = {}
    for entry in Q:
        strQ[str(entry)] = Q[entry]
    with open(filename, "w+") as writer:
        writer.write(json.dumps(strQ))
    writer.close()

# readQ(): read Q table from file
def readQ(filename):
    with open(filename) as f:
        data = f.read().replace("\n", " ")
    f.close()
    strQ = json.loads(data)
    Q = {}
    for entry in strQ:
        Q[literal_eval(entry[1:-1])] = strQ[entry]
    return Q

### Training

In order to train our three different player types, we were able to use the same trainQ function. The difference is the epsilonGreedy function, which we passed as an argument to trainQ and varied depending on whether the player is random, basic, or counter. Respectively, the epsilonGreedy functions are named random_epsilonGreedy, basic_epsilonGreedy, and counter_epsilonGreedy. This allows us to handle a player’s decisions before they use their Q tables and eventually converge on similar values. The difference between the players’ epsilon greedy functions, as explained in the previous section, has to do with the decisions they make during the exploration phase before epsilon decays enough to begin consulting the Q table.

In [7]:
# trainQ(): trains player for nRepetitions using the provided epsion greedy function
def trainQ(nRepetitions, epsilonDecayRate, learningRate, epsilonGreedyF):
    Q = {}
    rho = learningRate
    epsilon = 1.0
    deck = Deck.Deck()
    hand = Deck.Hand()
    dealer = Deck.Hand()
    
    win = 0
    loss = 0
    bust = 0
    draw = 0

    for n in range(nRepetitions):
        epsilon *= epsilonDecayRate
        step = 0
        done = False
        
        hand.add(deck.pop_card())
        hand.add(deck.pop_card())
        deck.pop_card()
        dealer.add(deck.pop_card())

        while not done:
            step += 1
            state = hand_to_state(hand)
            move = epsilonGreedyF(Q, epsilon, state, dealer, deck)
            makeMove(hand, move, deck)
            newState = hand_to_state(hand)         

            if state not in Q:
                Q[(state, move)] = 0

            if end_of_hand(hand, move):  # player stands or busts
                play_dealer(dealer, deck)
                if hand.bust():
                    penalty = 0.5 * (hand.get_value()-21)
                    Q[(state, move)] += rho * (-penalty - Q[(state, move)])
                    bust += 1
                elif winner(hand, dealer):
                    Q[(state, move)] += rho * (1 + Q[(state, move)])
                    win += 1
                elif winner(dealer, hand):
                    penalty = 0.03 * (dealer.get_value() - hand.get_value())
                    Q[(state, move)] += rho * (Q[(state, move)] - penalty)
                    loss += 1
                else:
                    draw += 1
                    
                done = True
                hand.clear()
                dealer.clear()

            if step > 1:
                Q[(oldState, oldMove)] += rho * (Q[(state, move)] - Q[(oldState, oldMove)])

            oldState, oldMove = state, move
            state = newState
    return Q, (win, loss, bust, draw)


# moveFromQ(): makes move based on Q table
def moveFromQ(Q, state):
    valid = validMoves()
    qVal = -1*sys.float_info.max
    move = ""
    for v in valid:
        curr = (state, v)
        if Q.get(curr, 0) > qVal:
            qVal = Q.get(curr, 0)
            move = v 
    return move

The random epsilon greedy function is simple: as its name implies, any value less than epsilon will result in the player making a completely random move, either hitting or standing regardless of the value of their hand. As epsilon decays, the random player’s choices are gradually shifting towards non-random choices based off its previous reinforcements in the Q table.

In [8]:
# random_epsilonGreedy(): epsilon greedy function for random player
def random_epsilonGreedy(Q, epsilon, state, dealer, deck):
    valid = validMoves()
    if np.random.uniform() < epsilon:
        return valid[random.randint(0, len(valid)-1)]
    else:
        Qs = np.array([Q.get((state, move), 0) for move in valid])
        return valid[np.argmax(Qs)]

The basic player’s greedy function is different, it first reads a matrix containing the statistically best moves to make for different hand combinations. It uses this matrix to base its moves at first and then shifts towards relying solely on its Q table. The idea behind providing the basic player with the strategy matrix is to give it an advantage compared to the random player and seeing how this affects the final results and if they deviate from the original strategy. 

In [9]:
# basic_epsilonGreedy(): epsilon greedy function for basic player
def basic_epsilonGreedy(Q, epsilon, state, dealer, deck):
    valid = validMoves()
    decisions = readBasicMatrix("basic_matrix.txt")
    lookupTuple = state_dealer_tuple(state, dealer)
    if np.random.uniform() < epsilon:
        if lookupTuple in decisions:
            return decisions[lookupTuple]
        else:
            return valid[random.randint(0, len(valid)-1)]
    else:
        Qs = np.array([Q.get((state, move), 0) for move in valid])
        return valid[np.argmax(Qs)]
 
    return

In [10]:
'''
Basic Strategy Matrix:

2 3 4 5 6 7 8 9 10 A
(8,0) (9,0) (10,0) (11,0) (12,0) (13,0) (14,0) (15,0) (16,0) (17,0) (13,1) (14,1) (15,1) (16,1) (17,1) (18,1) (19,1)
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit  
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand 
''';

Finally, the counter’s epsilon greedy function is more complex but similar in nature to the basic player’s function. The counter bases most of its moves off the same strategy matrix as the basic player, however, it makes a choice on specific moves that gives it a slight advantage when the count is at a certain level. For example, a basic player would hit on a play of 16 vs. 9, however, a card counter would stand if the count is at +5 or higher. 

In [11]:
# counter_epsilonGreedy(): epsilon greedy function for card counter
def counter_epsilonGreedy(Q, epsilon, state, dealer, deck):
    valid = validMoves()
    basic_decisions = readBasicMatrix("basic_matrix.txt")
    counter_decisions = readBasicMatrix("counter_matrix.txt")
    lookupTuple = state_dealer_tuple(state, dealer)
    if np.random.uniform() < epsilon:
        if lookupTuple in counter_decisions:
            decision = counter_decisions[lookupTuple]
            if decision == "Hit" or decision == "Stand":
                return decision
            else:
                print(lookupTuple)
                print("|" + str(decision) + "|")
                print(deck.true_count())
                if deck.true_count() >= int(decision):
                    if basic_decisions[lookupTuple] is "Stand":
                        return "Hit"
                    else:
                        return "Stand"
        elif lookupTuple in basic_decisions:
            return basic_decisions[lookupTuple]
        else:
            return valid[random.randint(0, len(valid)-1)]
    else:
        Qs = np.array([Q.get((state, move), 0) for move in valid])
        return valid[np.argmax(Qs)]
    
    return

In [12]:
'''
Card Counter Matrix:

2 3 4 5 6 7 8 9 10 A
(8,0) (9,0) (10,0) (11,0) (12,0) (13,0) (14,0) (15,0) (16,0) (17,0) (13,1) (14,1) (15,1) (16,1) (17,1) (18,1) (19,1)
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 4   2   0   -1   -1   Hit   Hit   Hit   Hit   Hit 
 -1   -2   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit   4   Hit 
 Stand   Stand   Stand   Stand   Stand   Hit   Hit   5   0   Hit 
 Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit  
 Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Stand   Stand   Hit   Hit   Hit 
 Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand   Stand 
''';

As far as actually using the Q-tables in gameplay, we needed a way to avoid training a player every time we wanted them to play games. Due to the complexity of the game and the various possible states, we needed to train each player for enough iterations to make sure they saw enough states. The states we really cared about were the ones containing 0-2 Aces, since anything more than that is uncommon enough that it didn’t matter much. This lead to about 23*2*3 = 138 (hands*hit/stand*1-2 Aces) different states that we wanted to catch. To accomplish this, we ran every player with 10,000 iterations, then saved their Q-tables as a .json file that could easily be read into a dictionary when we wanted to run each player. 

### Betting Strategies

Now we will take a more in-depth look at the betting strategies we used for each function. As stated earlier, the random betting was simply a random scalar multiple of the table minimum to keep things simple, allowing the random player to bet up to seven times the table minimum to occasionally step outside the bet spread.

In [13]:
# randomBet(): makes a bet for random player
def randomBet(tableMin, won, bet):
    return random.randint(1, 6) * tableMin

The basic player was a bit more controlled. If they won they up their bet by a random multiple of 5, and if they lost they reduce it. To keep them from dipping below the table min and from betting too far above the bet spread, we use min and max functions to control the bounds of the bet.

In [14]:
# basicBet(): makes a bet for basic player
def basicBet(tableMin, won, bet):
    if won:
        return min(bet + (5 * (random.randint(1, 5))), 7*tableMin)
    else:
        return max(bet - (5 * (random.randint(1, 5))), tableMin)

The counter bet is far more complex since the counter has more information, as well as multiple different betting strategies. The strategies employed by various counters were obvious, basic, penetration, camo, and all-in. 

In [15]:
# counterBet(): makes a bet for card counting player (defaults to basic, else calls appropriate method)
def counterBet(tableMin, count, deck, bet, won, counterType):    
    if counterType is "obvious":
        return obviousBet(tableMin, count)
    elif counterType is "penetration":
        return penetrationBet(tableMin, count, deck, bet, won)
    elif counterType is "camo":
        return camouflageBet(tableMin, count, deck, bet, won)
    elif counterType is "allin":
        return allInBet(tableMin, count, deck, bet, won)
    else:
        return basicCounterBet(tableMin, count)

The obvious bet was a control betting scheme, where the counter bets a fixed value directly correlated to the table count. This way, we could have a maximum correlation value to test later in our betting analysis.

In [16]:
# obviousBet(): makes a bet for card counting player (obvious)
def obviousBet(tableMin, count):
    if count < 1:
        return tableMin
    elif count >= 1 and count < 5:
        return tableMin*(2**(count-1))
    else:
        return tableMin*12

The basic counter bets some variation of the table min depending on the +/-/0 value of the table count to introduce some randomness, while still betting off the count to a certain degree. 

In [17]:
# basicCounterBet(): makes a bet for card counting player (basic)
def basicCounterBet(tableMin, count):
    if count < 0:
        return tableMin + 5*random.randint(0, 3)
    elif count == 0:
        return tableMin + 10*random.randint(0, 3)
    else:
        return tableMin + round(count)*5*random.randint(0, 3)

The penetration, camo, and all-in bets are a bit more sophisticated, all designed to conceal the counters intentions from the casino. The term penetration in blackjack refers to how deep into the “shoe” the cards have been dealt, where the shoe is the shuffled combination of all decks being used. For a 6-deck game, which is what we use, the shoe contains up to 6 decks at a time. As the shoe becomes more penetrated, the true count of the table becomes more informative for the counter. We found a good penetration value to be `0.6`, meaning 60% of the cards in the shoe have been dealt out already. The player uses the same progressive betting pattern as the basic player, all the while maintaining the count of the deck, until the shoe has been penetrated far enough. It then begins using the basic counter bet mentioned previously to make decisions off the count. In a real-life game, players can see the discard tray used by the dealer allowing them to estimate the penetration, so it wasn’t unreasonable for us to provide this information to the player. This betting scheme hides the correlation between a player’s bets and the table count, since 60% of the game will be played without the count.

In [18]:
# penetrationBet(): makes a bet for card counting player (pentration)
def penetrationBet(tableMin, count, deck, bet, won):
    if deck.penetration() < 0.6:
        return basicBet(tableMin, won, bet)
    else:
        return basicCounterBet(tableMin, count)

The camo and all-in bets were designed to hide from the house in more clever ways. Our camo betting scheme takes into account that most players will not raise their bet if they just lost a hand, as well as the fact that most players tend not to drastically spike their bets. It uses the penetration bet to get an initial betting value. It then adjusts this value accordingly if the counter lost the last hand and if the penetration bet provides too high of a spike. This allows the counter to still bet off the table count, but to conceal themselves more effectively.

In [19]:
# camouflageBet(): makes a bet for card counting player (camouflage)
def camouflageBet(tableMin, count, deck, bet, won):
    initialBet = penetrationBet(tableMin, count, deck, bet, won)
    if won is False:
        if initialBet == tableMin:
            return tableMin
        else:
            bestBet = min(bet + round(((initialBet-bet)/2) / 5)*5, round(((initialBet-tableMin)/2) / 5)*5)
            return max(bestBet, tableMin)
    elif won is None:
        return bet
    else:
        return initialBet

Lastly, the all-in bet is the closest thing to gambling that the counter does. It is not uncommon in casino games for players to bet the rest of their bankroll (a term used to describe the amount of money a player is willing to spend over the course of N-games) on the last round of a game before going home. This type of activity will always be seen as a spike, but casinos will tend to ignore it if the player leaves the table afterwards. The all-in counter uses penetration to wait for 75% of the shoe to be dealt into play. In the event that the shoe is 75% penetrated and the table count goes above 7, the counter will bet half their bankroll, which we have allotted at $100,000, then they will leave the table. If those conditions are not met, they play the same as a basic player, never playing off the table count. This allows a counter one chance to win big, since a high count indicates a good chance of getting blackjack, and over 100 games they will walk out with big winnings.

In [20]:
# allInBet(): goes all in if the count is +7 or higher
def allInBet(tableMin, count, deck, bet, won):
    if deck.penetration() < 0.75:
        return basicBet(tableMin, won, bet)
    else:
        if count > 7:
            return 50000
        else:
            return basicBet(tableMin, won, bet)

### Betting Analysis

To run a betting analysis, we needed a way to pass game data to the analysis program. We decided not to have them run in time with each other for the sake of simplicity, and planned to add concurrency later if we saw fit, but never got around to it. Instead, we had the `playGames()` function output all relevant data about each game to a file. This data included player type, whether the player won that game or not,  player bet, table count, player and dealer hands, and penetration value. Only some of this data is used in analysis, while all of it was used for our own purposes to see what our players were doing. Our analysis focuses on player bet, table count, and if they `won_last_game`, a value retrieved with a simple `i-1` indexing of the table for all `i > 0`.

In [21]:
# playGames(): plays numGames games using the provided Q table and betting function
def playGames(Q, numGames, betF, counterType=None):
    tableMin = 10
    bet = tableMin
    playerBalance = 100000
    won_last_game = False
    deck = Deck.Deck()
    deck.shuffle()
    player = Deck.Hand()
    dealer = Deck.Hand()
    
    win = 0
    loss = 0
    bust = 0
    draw = 0
    
    for n in range(numGames):
     
        if counterType is not None:
            bet = betF(tableMin, deck.true_count(), deck, bet, won_last_game, counterType)
        else:
            bet = betF(tableMin, won_last_game, bet)
   
        playerBalance -= bet
        count_at_bet = deck.true_count()
        done = False
        print("-------------------------------")
        print("Game number " + str(n+1))
        print("Running count is " + str(deck.count))
        print("Table count is " + str(count_at_bet))
        print("Player bets $" + str(bet))
        player.add(deck.pop_card())     #deal to player
        player.add(deck.pop_card())     #deal to player
        dealer.add(deck.pop_card())     #give dealer card
        print("Player hand: ")
        print(player, end="\n\n")
        print("Dealer hand: ")
        print(dealer, end="\n\n")
        
        while not done:
            move = moveFromQ(Q, hand_to_state(player))
            makeMove(player, move, deck)
            print("Player decides to " + move)
            print("Player hand: ")
            print(player, end="\n\n")
            if end_of_hand(player, move):  # player stands or busts
                play_dealer(dealer, deck)
                if player.bust():
                    print("Player Busted!")
                    print("Player: " + str(player.get_value()))
                    print("Dealer: " + str(dealer.get_value()))
                    won_last_game = False
                    bust += 1
                elif winner(player, dealer):
                    if player.blackjack():
                        print("BLACKJACK!")
                        playerBalance += bet*2.5
                    else:
                        print("Player Won!")
                        playerBalance += bet*2
                    print("Player: " + str(player.get_value()))
                    print("Dealer: " + str(dealer.get_value()))
                    won_last_game = True
                    win += 1
                elif winner(dealer, player):
                    print("Dealer won...")
                    print("Player: " + str(player.get_value()))
                    print("Dealer: " + str(dealer.get_value()))
                    won_last_game = False
                    loss += 1
                else:
                    print("Draw!")
                    print("Player: " + str(player.get_value()))
                    print("Dealer: " + str(dealer.get_value()))
                    playerBalance += bet
                    won_last_game = None
                    draw += 1
                    
                done = True
                writeGameData(counterType, won_last_game, bet, count_at_bet, hand_to_state(player), hand_to_state(dealer), deck.penetration())
                player.clear()
                dealer.clear()
                print()
                   
        if(counterType is "allin"):
            if bet > 500:
                print("Big Spender!!! Balance:", playerBalance)
                break;
                     
    return playerBalance, (win, loss, bust, draw)


# writeGameData(): write game data to file
def writeGameData(counterType, won, bet, count, state, dealer, penetration):
    with open("game_data.txt", "a+") as f:
        f.write(str(won) + "\t" + str(bet) + "\t" + str(count) + "\t" + str(state) + "\t" + str(dealer) + "\t" + str(penetration) + "\t" + str(counterType) + "\n")
    f.close()

Our whole betting analysis is run as a script over the entirety of the game data for a certain player, returning the Pearson coefficient and if a flag was spiked or not. We wrote a method that calculates the Pearson coefficient for table count and player bet, though we realized later that this method is already included in the numpy library. However, both methods run the same, so it was good to know that we were coding the correct method. The formula is readily available online and was just translated to python code.

In [22]:
# pearson(): finds the correlation coefficient between X and Y
def pearson(X, Y):
    N = len(X)
    if N != len(Y):
        return None
    sqrX = [x**2 for x in X]
    sqrY = [y**2 for y in Y]
    XY = []
    for i in range(N):
        XY.append(X[i]*Y[i])
    r = (N*sum(XY) - sum(X)*sum(Y)) / math.sqrt((N*sum(sqrX)-sum(X)**2)*(N*sum(sqrY)-sum(Y)**2))
    return r

Our spike analysis was a bit more interesting. It begins by checking if the current bet is too much of a jump from the last bet. Because of the definition of a spike, this is the most important factor. Then it checks if the last game was won or not, allowing players to spike if they won the last game. If both of these checks fail, it does one last check of the table count. If the table count is above 2, a flag is thrown. We added this final check because the whole point of this is to keep players from making money. If a player spikes and their betting is correlated with the count, we only really care when the count is high enough to hurt the casino. This could be modified for any constraints though, disregarding the table count and throwing a flag any time someone deviates from their bet too much.

In [23]:
# spikeFlag(): checks that a user is not increasing their bet by a large margin (spiking)
def spikeFlag(x, y, won, tableMin, filename):
    for i in range(1, len(y)):
        if (y[i] - y[i-1]) > 4*tableMin:
            if won[i-1] == "False":
                if x[i] > 2:
                    return "Flag spike! for game: " + str(i)
    return "none"

Neither of these methods is effective on their own, and are best used together as a way to inform casino management that someone may be counting cards.

In [24]:
# plot(): plots the correlation graph
def plot(x, y):
    plt.plot(x, y, 'ro')
    plt.axis([-15, 15, 0, 500])
    plt.show()
    
# runAnalysis(): read game data into lists and run analysis methods
def runAnalysis(filename):
    x = []
    y = []
    won = []
    with open(filename) as f:
        for line in f:
            vals = line.split()
            won.append(vals[0])
            x.append(float(vals[2]))
            y.append(float(vals[1]))
    f.close()
    print(pearson(x, y))
    print(np.corrcoef(x, y)[0, 1])
    print(spikeFlag(x, y, won, 10, filename))
    plot(x, y)

## Results

Overall, our final results turned out to be much different than we expected. As mentioned earlier, all of our players ended up converging on similar values. We calculated performance off of a Win/Loss/Bust/Draw ratio as well as the final balance in each player’s bankroll. In order to get a good sample size we ran 100 games over 100 iterations for each player to get 10,000 total games, averaging everything at the end. This way, we still got a good volume of games to analyze, but all of our numbers came out neatly out of 100. Our final results for each player were as follows:

| Player                | Win   | Loss  | Bust | Draw | Balance   |
|-----------------------|-------|-------|------|------|-----------|
| Basic                 | 39.58 | 54.33 | 0.00 | 6.09 | 99662.30  |
| Counter               | 39.27 | 53.98 | 0.00 | 5.76 | 101150.52 |
| Counter (All-In)      | 46.00 | 50.00 | 0.00 | 4.00 | 100137.50 |
| Counter (Basic)       | 48.00 | 48.00 | 0.00 | 4.00 | 99970.00  |
| Counter (Camouflage)  | 40.00 | 54.00 | 0.00 | 6.00 | 99692.50  |
| Counter (Obvious)     | 40.30 | 54.65 | 0.00 | 5.05 | 99710.62  |
| Counter (Penetration) | 40.30 | 53.50 | 0.00 | 6.20 | 99767.50  |
| Random                | 41.20 | 52.99 | 0.00 | 5.81 | 99768.40  |

As mentioned earlier, all of our players stay within the bounds of what we deem acceptable for average blackjack results. The section for Bust is zero across all columns because our players actually busted so infrequently that over 10,000 games they average to incredibly small values. 

It’s also important to note that since the betting can vary so much for each player, the final average balance varies between each 10,000 runs. Fortunately, the balance is still indicative of player performance, since our counters who do not try to hide from the casino make more money (better yet, lose less money in most cases) than the basic and random players. Our counters that attempted to hide from the casino did so at the expense of not making as much money, which is a fantastic result. This proves that betting strategies that attempt to hide are not as effective at making more money since they are forced to ignore the table count at times, and as a corollary it shows that playing off the table count can win you more money than you lose. 

It’s also exciting to see that our all-in player came out with positive winnings. While this doesn’t mean that the player won big every 100 games, it does indicate that in at least one round of 100, the all-in player successfully played a large bet on a high count and won blackjack on the next hand. 

Furthermore, our betting analysis doesn’t catch the attempting to hide. It will occasionally flag the penetration and camo players with spikes, but since they wait to bet big, their correlation is low enough to be ignored. On the other side of the game, our basic player never got flagged and our random player only occasionally. When both players were analyzed for correlation, they were completely negligible, sitting at around +/-0.03, indicating zero correlation between table count and bet, as they should.

In [25]:
import Deck
import numpy as np
import random
import sys
import fileIO

# fullRun(): allows you to specify how many games to run
def fullRun(playerType, numGames, betF, counterType=None):
    Q = fileIO.readQ(playerType + "Q.json")
    balance, results = playGames(Q, numGames, betF, counterType)
    if counterType is not None:
        playerType += "_" + counterType
    with open(playerType + '_results.txt', 'a+') as writer:
        writer.write(str(results[0]) + " " + str(results[1]) + " " + str(results[2]) + " " + str(results[3]) + " " + str(balance) + '\n')
    writer.close()

In [26]:
fullRun("basic", 10, basicBet)

-------------------------------
Game number 1
Running count is 0
Table count is 0.0
Player bets $10
Player hand: 
Jack of Spades has a value of 10.
5 of Spades has a value of 5.
Value of hand: 15

Dealer hand: 
7 of Hearts has a value of 7.
Value of hand: 7

Player decides to Stand
Player hand: 
Jack of Spades has a value of 10.
5 of Spades has a value of 5.
Value of hand: 15

Dealer won...
Player: 15
Dealer: 20

-------------------------------
Game number 2
Running count is 2
Table count is 2.0
Player bets $10
Player hand: 
8 of Diamonds has a value of 8.
10 of Clubs has a value of 10.
Value of hand: 18

Dealer hand: 
4 of Spades has a value of 4.
Value of hand: 4

Player decides to Stand
Player hand: 
8 of Diamonds has a value of 8.
10 of Clubs has a value of 10.
Value of hand: 18

Player Won!
Player: 18
Dealer: 13

-------------------------------
Game number 3
Running count is 1
Table count is 1.0
Player bets $35
Player hand: 
9 of Spades has a value of 9.
3 of Hearts has a value of

## Conclusion:

Our final results ended up being deeply satisfying. All of our players were trained within a few percent of expected, our counter programs that don’t hide their bets made more money, our camouflaged betters evaded the analysis program at the cost of losing money, and our betting analysis program successfully caught all players that we expected it to while ignoring the rest. However, there were many improvements that could have been made to this project. 

For starters, it would have been far more beneficial to add functionality for doubling-down and splitting, since those moves are important for effective betting and playing strategies. We simply just ran out of time with the rest of the functionality we wanted to implement. 

Additionally, we feel that our counter should have had a slightly better advantage in play over the random and basic players. While the results posted in this notebook indicate that this was the case for some counters, all of them were trained the same, so we would have expected less variation. We believe that better training values over more iterations, with more than 10,000 games played might have made this a bit better. However, we may have just ended up with the same results anyways. Again, we ran out of time to start messing around with the training that we felt it best to just leave it.

Another way we would have liked to improve this project would be to run our betting analysis as an A.I. rather than a script, training it on data from various games. This way it might be better at catching the counter quicker, and ignoring insignificant players. Better yet, if we trained it on camouflaged players, we believe that eventually it would be able to catch even people trying to hide from the casino. The last step we might add to this is to run both A.I.’s one-on-one, training each other as they go until the point where either counters are indistinguishable from normal players, or the analysis A.I can catch anybody who is counting regardless of how they play.

Unfortunately, all of this seems like a pipe dream. While we believe it could be useful if improved upon, we did some research into how casinos catch card-counters. It would seem that human factors and simply imposing rules on the game are effective enough to catch counters. Some of the human factors used by casinos involve watching the player’s actions: whether they’re signaling teammates, waiting to enter a game until the shoe is penetrated, even what they’re drinking, as alcohol (or lack thereof) can be an indicator that someone is counting. Furthermore, it seems easier to just impose rules on the game that make it impractical to count cards. The two biggest things a casino may do are maintaining a constant 6-deck shoe, always replenishing it with new decks, and forbidding mid-shoe play, making penetration betting impossible. 

So at the end of the day, it would seem that all of this just wouldn’t be used by a real casino. Especially since that would require the addition of computer vision to actually look at the cards on the table and maintain a running count as well as the player bets. Fortunately, this is exactly the type of program that could be employed by online and electronic gambling, where players flagged as counters get suspended from their accounts for X amount of time. Regardless of the usefulness of the software, it was still a valuable learning experience for our team, and if anything came out of it, now we all know how to count cards and beat the casino!

In [27]:
import io
import warnings
warnings.filterwarnings('ignore')

from IPython.nbformat import current
import glob
nbfile = glob.glob('Batres-Dennis-Nickels-Blackjack-AI.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Batres-Dennis-Nickels-Blackjack-AI.ipynb is 5068
